In [143]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
import sys

sys.path.append("..")
import pandas as pd 
import numpy as np 

import torch
import torchvision
import torch.nn as nn
from torchvision.ops.misc import MLP, Conv2dNormActivation
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim

from collections import OrderedDict
import math
from functools import partial
from typing import Any, Callable, Dict, List, NamedTuple, Optional

from src.processing import create_data_for_vision

# Create Data for Vision Transformer
### We will target Western Plateau for consistency 

In [145]:
df_train_ls, df_test_ls, features = create_data_for_vision.create_data_for_model()

Compiling Data for ADDI
Test Set Fraction 0.2000117792567289
train_shape (27166, 34)
test_shape (6792, 34)
Compiling Data for BELM
Test Set Fraction 0.2000117792567289
train_shape (27166, 34)
test_shape (6792, 34)
Compiling Data for COHO
Test Set Fraction 0.2000117792567289
train_shape (27166, 34)
test_shape (6792, 34)
Compiling Data for DELE
Test Set Fraction 0.2000117792567289
train_shape (27166, 34)
test_shape (6792, 34)
Compiling Data for ELMI
Test Set Fraction 0.2000117792567289
train_shape (27166, 34)
test_shape (6792, 34)
Compiling Data for GROV
Test Set Fraction 0.2000117792567289
train_shape (27166, 34)
test_shape (6792, 34)
Compiling Data for HART
Test Set Fraction 0.2000117792567289
train_shape (27166, 34)
test_shape (6792, 34)
Compiling Data for OLEA
Test Set Fraction 0.2000117792567289
train_shape (27166, 34)
test_shape (6792, 34)
Compiling Data for RAND
Test Set Fraction 0.2000117792567289
train_shape (27166, 34)
test_shape (6792, 34)


In [146]:
df_train_ls[0].keys()

Index(['t2m', 'sh2', 'd2m', 'r2', 'u10', 'v10', 'tp', 'mslma', 'orog', 'tcc',
       'asnow', 'cape', 'dswrf', 'dlwrf', 'gh', 'u_total', 'u_dir', 'new_tp',
       'day_of_year_cos', 'day_of_year_sin', 'target_error', 'elev', 'tair',
       'ta9m', 'td', 'relh', 'srad', 'pres', 'mslp', 'wspd_sonic',
       'wmax_sonic', 'wdir_sonic', 'precip_total', 'snow_depth'],
      dtype='object')

In [147]:
class MultiStationDataset(Dataset):
    def __init__(self, dataframes, target, features, past_steps, future_steps=1, nysm_vars=12):
        """
        dataframes: list of station dataframes like in the SequenceDataset
        target: same as SequenceDataset
        features: same as SequenceDataset
        sequence_length: SequenceDataset
        """
        self.dataframes = dataframes
        self.features = features
        self.target = target
        self.past_steps = past_steps
        self.future_steps = future_steps
        self.nysm_vars = nysm_vars

    def __len__(self):
        shaper = min([self.dataframes[i].values.shape[0] - (self.past_steps + self.future_steps) for i in range(len(self.dataframes))])
        return shaper

    def __getitem__(self, i):
        # this is the preceeding sequence_length timesteps
        x = torch.stack([torch.tensor(dataframe[self.features].values[i : i + self.past_steps + self.future_steps]) for dataframe in self.dataframes])
        # stacking the sequences from each dataframe along a new axis, so the output is of shape (batch, stations (len(self.dataframes)), past_steps, features)
        y = torch.stack([torch.tensor(dataframe[self.target].values[i + self.past_steps : i + self.past_steps + self.future_steps]) for dataframe in self.dataframes])
        # this is (batch, stations, future_steps)
        x[-self.future_steps:, :self.nysm_vars] = -999.0 # check that this is setting the right positions to this value
        return x, y

## Load Datasets

In [148]:
train_dataset = MultiStationDataset(df_train_ls, 'target_error', features, 8)

In [149]:
test_dataset = MultiStationDataset(df_test_ls, 'target_error', features, 8)

# Define Model 

In [150]:
class MLPBlock(MLP):
    """Transformer MLP block."""

    _version = 2

    def __init__(self, in_dim: int, mlp_dim: int, dropout: float):
        super().__init__(in_dim, [mlp_dim, in_dim], activation_layer=nn.GELU, inplace=None, dropout=dropout)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.normal_(m.bias, std=1e-6)

    def _load_from_state_dict(
        self,
        state_dict,
        prefix,
        local_metadata,
        strict,
        missing_keys,
        unexpected_keys,
        error_msgs,
    ):
        version = local_metadata.get("version", None)

        if version is None or version < 2:
            # Replacing legacy MLPBlock with MLP. See https://github.com/pytorch/vision/pull/6053
            for i in range(2):
                for type in ["weight", "bias"]:
                    old_key = f"{prefix}linear_{i+1}.{type}"
                    new_key = f"{prefix}{3*i}.{type}"
                    if old_key in state_dict:
                        state_dict[new_key] = state_dict.pop(old_key)

        super()._load_from_state_dict(
            state_dict,
            prefix,
            local_metadata,
            strict,
            missing_keys,
            unexpected_keys,
            error_msgs,
        )


class EncoderBlock(nn.Module):
    """Transformer encoder block."""

    def __init__(
        self,
        num_heads: int,
        hidden_dim: int,
        mlp_dim: int,
        dropout: float,
        attention_dropout: float,
        norm_layer: Callable[..., torch.nn.Module] = partial(nn.LayerNorm, eps=1e-6),
    ):
        super().__init__()
        self.num_heads = num_heads

        # Attention block
        self.ln_1 = norm_layer(hidden_dim)
        self.self_attention = nn.MultiheadAttention(hidden_dim, num_heads, dropout=attention_dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

        # MLP block
        self.ln_2 = norm_layer(hidden_dim)
        self.mlp = MLPBlock(hidden_dim, mlp_dim, dropout)

    def forward(self, input: torch.Tensor):
        torch._assert(input.dim() == 3, f"Expected (batch_size, seq_length, hidden_dim) got {input.shape}")
        x = self.ln_1(input)
        x, _ = self.self_attention(x, x, x, need_weights=False)
        x = self.dropout(x)
        x = x + input

        y = self.ln_2(x)
        y = self.mlp(y)
        return x + y


class Encoder(nn.Module):
    """Transformer Model Encoder for sequence to sequence translation."""

    def __init__(
        self,
        seq_length: int,
        num_layers: int,
        num_heads: int,
        hidden_dim: int,
        mlp_dim: int,
        dropout: float,
        attention_dropout: float,
        norm_layer: Callable[..., torch.nn.Module] = partial(nn.LayerNorm, eps=1e-6),
    ):
        super().__init__()
        # Note that batch_size is on the first dim because
        # we have batch_first=True in nn.MultiAttention() by default
        self.pos_embedding = nn.Parameter(torch.empty(1, seq_length, hidden_dim).normal_(std=0.02))  # from BERT
        self.dropout = nn.Dropout(dropout)
        layers: OrderedDict[str, nn.Module] = OrderedDict()
        for i in range(num_layers):
            layers[f"encoder_layer_{i}"] = EncoderBlock(
                num_heads,
                hidden_dim,
                mlp_dim,
                dropout,
                attention_dropout,
                norm_layer,
            )
        self.layers = nn.Sequential(layers)
        self.ln = norm_layer(hidden_dim)

    def forward(self, input: torch.Tensor):
        torch._assert(input.dim() == 3, f"Expected (batch_size, seq_length, hidden_dim) got {input.shape}")
        input = input + self.pos_embedding
        return self.ln(self.layers(self.dropout(input)))


class VisionTransformer(nn.Module):
    """Vision Transformer as per https://arxiv.org/abs/2010.11929."""

    def __init__(
        self,
        stations: int,
        past_timesteps: int,
        future_timesteps: int,
        patch_size: int = 16,
        num_layers: int = 6,
        num_heads: int = 8,
        hidden_dim: int = 128,
        mlp_dim: int = 768,
        dropout: float = 0.0,
        attention_dropout: float = 0.0,
        num_classes: int = 1,
        representation_size: Optional[int] = None,
        norm_layer: Callable[..., torch.nn.Module] = partial(nn.LayerNorm, eps=1e-6),
    ):
        super().__init__()
        self.future_timesteps = future_timesteps
        self.past_timesteps = past_timesteps
        self.stations = stations
        self.timesteps = future_timesteps + past_timesteps
        self.hidden_dim = hidden_dim
        self.mlp_dim = mlp_dim
        self.attention_dropout = attention_dropout
        self.dropout = dropout
        self.num_classes = num_classes
        self.representation_size = representation_size
        self.norm_layer = norm_layer

        seq_length = stations * (future_timesteps + past_timesteps)

        # Add a class token
        self.class_token = nn.Parameter(torch.zeros(1, 1, hidden_dim))
        seq_length += 1

        self.encoder = Encoder(
            seq_length,
            num_layers,
            num_heads,
            hidden_dim,
            mlp_dim,
            dropout,
            attention_dropout,
            norm_layer,
        )
        self.seq_length = seq_length

        heads_layers: OrderedDict[str, nn.Module] = OrderedDict()
        if representation_size is None:
            heads_layers["head"] = nn.Linear(hidden_dim, num_classes)
        else:
            heads_layers["pre_logits"] = nn.Linear(hidden_dim, representation_size)
            heads_layers["act"] = nn.Tanh()
            heads_layers["head"] = nn.Linear(representation_size, num_classes)

        self.heads = nn.Sequential(heads_layers)

        if hasattr(self.heads, "pre_logits") and isinstance(self.heads.pre_logits, nn.Linear):
            fan_in = self.heads.pre_logits.in_features
            nn.init.trunc_normal_(self.heads.pre_logits.weight, std=math.sqrt(1 / fan_in))
            nn.init.zeros_(self.heads.pre_logits.bias)

        if isinstance(self.heads.head, nn.Linear):
            nn.init.zeros_(self.heads.head.weight)
            nn.init.zeros_(self.heads.head.bias)

    def _process_input(self, x: torch.Tensor) -> torch.Tensor:
        n, h, w, c = x.shape
        torch._assert(h == self.stations, f"Wrong image height! Expected {self.image_size} but got {h}!")
        torch._assert(w == self.timesteps, f"Wrong image width! Expected {self.image_size} but got {w}!")

        # (n, hidden_dim, n_h, n_w) -> (n, hidden_dim, (n_h * n_w))
        x = x.reshape(n, h * w, self.hidden_dim)

        return x

    def forward(self, x: torch.Tensor):
        # Reshape and permute the input tensor
        x = self._process_input(x)
        n = x.shape[0]

        # Expand the class token to the full batch
        batch_class_token = self.class_token.expand(n, -1, -1)
        x = torch.cat([batch_class_token, x], dim=1)

        x = self.encoder(x)

        # Classifier "token" is the future prediction - we will probably just want to select just some of these variables.
        # x \in (batch, stations * timesteps + 1, num_classes = 1)
        x = x[:, -(self.stations * self.future_timesteps):, :] # this shape is (batch, stations, num_classes = 1)

        x = self.heads(x) # is a linear transformation from hidden_dim to 1
        
        return x # logically we are saying return one value for the each future timestep for each station (interpreted as error)


class AaronFormer(nn.Module):
    def __init__(self, 
                output_dim, 
                stations,
                past_timesteps=8,
                future_timesteps=8,
                patch_size=16,                 
                num_layers=12,
                num_heads=12,
                hidden_dim=768,
                mlp_dim=3072,
                dropout=0.0, 
                attention_dropout=0.0,
                ):
        super().__init__()

        self.encoder = VisionTransformer(
                stations=stations,
                past_timesteps=past_timesteps,
                future_timesteps=future_timesteps,
                patch_size=patch_size,
                num_layers=num_layers,
                num_heads=num_heads,
                hidden_dim=hidden_dim,
                mlp_dim=mlp_dim,
                num_classes=output_dim,
                dropout=dropout,
                attention_dropout=attention_dropout
            )

    def forward(self, x):
        x = self.encoder(x)
        return x

# Define Model Parameters

In [151]:
model = AaronFormer(output_dim=1, 
stations = len(df_train_ls), 
past_timesteps=8,
future_timesteps=8,
patch_size=16,                 
num_layers=12,
num_heads=12,
hidden_dim=768,
mlp_dim=3072,
dropout=0.0, 
attention_dropout=0.0,
)

In [152]:
EPOCHS = 3
BATCH_SIZE = 10
LEARNING_RATE = 2e-5

In [153]:
# Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# Cross Entropy Loss
loss_func = nn.MSELoss()
# Use GPU if available  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if torch.cuda.is_available():
    model.cuda() 

In [154]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 

In [155]:
def train_model(data_loader, model, loss_function, optimizer, device, epoch):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    ddp_loss = torch.zeros(2).to(device)

    for batch_idx, (X, y) in enumerate(data_loader):
        # Move data and labels to the appropriate device (GPU/CPU).
        X, y = X.to(device), y.to(device)

        # Forward pass and loss computation.
        output = model(X)
        loss = loss_function(output, y)

        # Zero the gradients, backward pass, and optimization step.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the total loss and the number of processed samples.
        total_loss += loss.item()
        ddp_loss[0] += loss.item()
        ddp_loss[1] += len(X)

    # Synchronize and aggregate losses in distributed training.
    dist.all_reduce(ddp_loss, op=dist.ReduceOp.SUM)

    # Compute the average loss for the current epoch.
    avg_loss = total_loss / num_batches

    # Print the average loss on the master process (rank 0).
    if rank == 0:
        print("Train Epoch: {} \tLoss: {:.6f}".format(epoch, ddp_loss[0] / ddp_loss[1]))

    return avg_loss



def test_model(data_loader, model, loss_function, device):
    # Test a deep learning model on a given dataset and compute the test loss.

    num_batches = len(data_loader)
    total_loss = 0

    # Set the model in evaluation mode (no gradient computation).
    model.eval()

    # Initialize an array to store loss values.
    ddp_loss = torch.zeros(3).to(device)

    with torch.no_grad():
        for batch_idx, (X, y) in enumerate(data_loader):
            # Move data and labels to the appropriate device (GPU/CPU).
            X, y = X.to(device), y.to(device)

            # Forward pass to obtain model predictions.
            output = model(X)

            # Compute loss and add it to the total loss.
            total_loss += loss_function(output, y).item()

            # Update aggregated loss values.
            ddp_loss[0] += F.mse_loss(output, y, reduction="sum").item()
            ddp_loss[2] += len(X)

    # Calculate the average test loss.
    avg_loss = total_loss / num_batches

    # Synchronize and aggregate loss values in distributed testing.
    dist.all_reduce(ddp_loss, op=dist.ReduceOp.SUM)

    # Print the test loss on the master process (rank 0).
    if rank == 0:
        test_loss = ddp_loss[0] / ddp_loss[2]
        print(
            "Test set: Average loss: {:.4f}\n".format(test_loss)
        )

    return avg_loss

In [156]:
for ix_epoch in range(1, EPOCHS + 1):
    print("Epoch", ix_epoch)
    train_loss = train_model(
        train_loader, model, loss_func, optimizer, device, ix_epoch
    )
    test_loss = test_model(test_loader, model, loss_func, device)
    scheduler.step()
    print()

Epoch 1


AttributeError: 'VisionTransformer' object has no attribute 'image_size'